In [ ]:
from utils import CNC
from types import SimpleNamespace

In [ ]:
args = SimpleNamespace()
args.file_path = "./data/data.csv"
args.model = "logistic_regression"
args.future_steps = 1
args.window_size = 30
args.test_size = 0.2
args.seed = 0
cnc = CNC(args)

In [ ]:
cnc.pre_process()

In [ ]:
cnc.train()

In [ ]:
cnc.evaluate()

In [ ]:
cnc.visualize()